In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
import configparser
import os
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk
import plotly_express as px
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from numpy.linalg import norm
from scipy.spatial.distance import pdist
import plotly_express as px
import seaborn as sns; sns.set()
import configparser
config = configparser.ConfigParser()
config.read("../../../env-sample.ini")
data_home = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data'
output_dir = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/output'
local_lib = '/Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib'
import sys
sys.path.append(local_lib)
from textparser import TextParser

clip_pats = [
    r"\*\*\*\s*START OF",
    r"\*\*\*\s*END OF"
]

# All are 'chap'and 'm'
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
ohco_pat_list = [
    (5200,   rf"^\s*CHAPTER\s+{roman}\s*$"), #Metamorphosis
    (7849,   rf"^\s*{roman}\s*$"), #The Trial
    (6969,  rf"^\s*LETTER .* to .*$"), # The Castle
    (6262,   rf"^CHAPTER\s+{roman}$"), # Amerika
    (6161,   rf"^CHAPTER\s+\d+$"), # The Judgement
    (6060,   rf"^Chapter\s+\d+$"), # Dearest Father
    (6363,  rf"^Chapter\s+\d+$"), # In the Penal colony
    (6464,   rf"^CHAPTER\s+\d+$"), # The Hunger Artist
    (6565, rf"^\s*CHAPTER\s+{roman}\."), # The Jackals and Arabs
    (6666, rf"^\s*CHAPTER\s+{roman}\s*$"), # A Country Doctor
    (6767, rf"^\s*CHAPTER\s+{roman}\s*$"), # An Imperial Message
    (5959,  rf"^(?:ETYMOLOGY|EXTRACTS|CHAPTER)"), # A report for an Academy
    (5858,  rf"^\s*CHAPTER\s+{roman}\.\s*$"), # The Great Wall of China
    (5757, rf"^\s*{roman}\.\s*$"), # The Hunter Gracchus
    (5656,  rf"^\s*{roman}\. .*$"), # Up in the Gallery
    (5555, rf"^CHAPTER\s+{roman}\.?$"), # Before the Law
    (5454, rf"^\s*[A-Z,;-]+\.\s*$"), # Josephine the Songstress
    (5353,  rf"^CHAPTER "), # The Burrow
    (5252, rf"^CHAPTER\s+{roman}\.\s*$"), # Blumfeld
    (23532, rf"Chapter\s+{roman}") # Meditation
]
chapter_regexes = [
    (5200,   rf"^\s*{roman}\s*$"),
    (7849,   rf"^\s*Chapter\s+(?:One|Two|Three|Four|Five|Six|Seven|Eight|Nine|Ten)\s*$"),
    (6969,   rf"^\s*\d+\s*$"),
    (6262,   rf"^\s*\d+\s*$"),
    (6161,   "NOCHAPTERS"),
    (6060,   "NOCHAPTERS"),
    (6363,   "NOCHAPTERS"),
    (6464,   "NOCHAPTERS"),
    (6565,   "NOCHAPTERS"),
    (6666,   "NOCHAPTERS"),
    (6767,   "NOCHAPTERS"),
    (5959,   "NOCHAPTERS"),
    (5858,   "NOCHAPTERS"),
    (5757,   "NOCHAPTERS"),
    (5656,   "NOCHAPTERS"),
    (5555,   "NOCHAPTERS"),
    (5454,   "NOCHAPTERS"),
    (5353,   "NOCHAPTERS"),
    (5252,   "NOCHAPTERS"),
    (23532,  rf"^(Children on the country road|Unmasking a con artist|The Sudden Walk|Resolutions|The trip to the mountains|The Bachelor's Misfortune|The Merchant|Distracted Looking Out|The Way Home|The Passers-by|Passenger|Dresses|The rejection|Food for thought for gentlemen riders|The Alley Window|Desire to become an Indian|The Trees|Unhappiness)$")  # Poem title on line 1
]
ohco_pat_list = chapter_regexes
source_files = f'{data_home}'
source_file_list = sorted(glob(f"{source_files}/*.*"))

book_data = []
for source_file_path in source_file_list:
    # Get the filename only, e.g. 'pg5353.txt'
    filename = os.path.basename(source_file_path)
    # Extract the numeric ID from the filename (remove 'pg' and '.txt')
    book_id = int(filename.replace('pg', '').replace('.txt', ''))
    # Use filename (without extension) as a raw title (optional: clean further)
    book_title = filename.replace('.txt', '').replace('_', ' ')
    # Append a tuple of (book_id, path, title)
    book_data.append((book_id, source_file_path, book_title))
# Convert to DataFrame
LIB = pd.DataFrame(book_data, columns=['book_id', 'source_file_path', 'raw_title']) \
        .set_index('book_id') \
        .sort_index()
book_titles = {
    5200: "Metamorphosis",
    7849: "The Trial",
    6969: "The Castle",
    6262: "Amerika",
    6161: "The Judgement",
    6060: "Dearest Father",
    6363: "In the Penal Colony",
    6464: "The Hunger Artist",
    6565: "The Jackals and Arabs",
    6666: "A Country Doctor",
    6767: "An Imperial Message",
    5959: "A Report for an Academy",
    5858: "The Great Wall of China",
    5757: "The Hunter Gracchus",
    5656: "Up in the Gallery",
    5555: "Before the Law",
    5454: "Josephine the Songstress",
    5353: "The Burrow",
    5252: "Blumfeld",
    23532: "Meditation"
}
book_titles = {f'pg{key}': value for key, value in book_titles.items()}
try:
    LIB['author'] = 'KAFKA, FRANZ'
    LIB['title'] = LIB.raw_title.replace(book_titles).str.upper()
    LIB = LIB.drop('raw_title', axis=1)
except AttributeError:
    pass
LIB['chap_regex'] = LIB.index.map(pd.Series({x[0]:x[1] for x in ohco_pat_list}))
LIB
# This cell takes 16 seconds to run
def tokenize_collection(LIB):

    clip_pats = [
        r"\*\*\*\s*START OF",
        r"\*\*\*\s*END OF"
    ]

    books = []
    for book_id in LIB.index:

        # Announce
        print("Tokenizing", book_id, LIB.loc[book_id].title)

        # Define vars
        chap_regex = LIB.loc[book_id].chap_regex
        ohco_pats = [('chap', chap_regex, 'm')]
        src_file_path = LIB.loc[book_id].source_file_path

        # Create object
        text = TextParser(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats, use_nltk=True)
        # text = TextImporter(src_file_path, ohco_pats=ohco_pats, clip_pats=clip_pats) 

        # Define parameters
        text.verbose = True
        text.strip_hyphens = True
        text.strip_whitespace = True

        # Parse
        text.import_source().parse_tokens();

        # Name things
        text.TOKENS['book_id'] = book_id
        text.TOKENS = text.TOKENS.reset_index().set_index(['book_id'] + text.OHCO)

        # Add to list
        books.append(text.TOKENS)
        
    # Combine into a single dataframe
    CORPUS = pd.concat(books).sort_index()

    # Clean up
    del(books)
    del(text)
        
    print("Done")
        
    return CORPUS
CORPUS = tokenize_collection(LIB)

Tokenizing 5200 METAMORPHOSIS
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5200.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^\s*[IVXLCM]+\s*$
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5252 BLUMFELD
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5252.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
line_str chap_str
Index(['chap_str'], dtype='object')
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by NLTK model
Parsing OHCO level 3 token_num by NLTK model
Tokenizing 5353 THE BURROW
Importing  /Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/KafkaFinal/data\pg5353.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone NOCHAPTERS
li

c:\Users/muhur/OneDrive/Desktop/Muhurto/Data Science Grad School/DS5001/DS5001-2025-01-R/lessons/lib\textparser.py:132: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  div_lines = self.TOKENS[src_col].str.contains(div_pat, regex=True, case=True)


Done


In [2]:
CORPUS = CORPUS[CORPUS.term_str != '']
CORPUS['pos_group'] = CORPUS.pos.str[:2]
CORPUS

pos_tuple  pos token_str  \
book_id chap_id para_num sent_num token_num                                  
5200    1       1        0        0               (One, CD)   CD       One   
                                  1          (morning,, NN)   NN  morning,   
                                  2             (when, WRB)  WRB      when   
                                  3           (Gregor, NNP)  NNP    Gregor   
                                  4            (Samsa, NNP)  NNP     Samsa   
...                                                     ...  ...       ...   
23532   10      39       0        27         (upstairs, JJ)   JJ  upstairs   
                                  28              (and, CC)   CC       and   
                                  29               (go, VB)   VB        go   
                                  30               (to, TO)   TO        to   
                                  31           (sleep., VB)   VB    sleep.   

                                             term_str pos_group  
book_id chap_id para_num sent_num token_num                      
5200    1       1        0        0               one        CD  
                                  1           morning        NN  
                                  2              when        WR  
                                  3            gregor        NN  
                                  4             samsa        NN  
...                                               ...       ...  
23532   10      39       0        27         upstairs        JJ  
                                  28              and        CC  
                                  29               go        VB  
                                  30               to        TO  
                                  31            sleep        VB  

[423144 rows x 5 columns]

In [3]:
#CORPUS.to_csv('CORPUS.csv')